# Part I. ETL Pipeline for Pre-Processing the Files

## 1. Import Python packages 

In [1]:
# Import Python packages 
import re
import os
import csv
import glob
import json

import cassandra
import pandas as pd
import numpy as np
from tqdm import tqdm

## 2. Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data/'
print(f"Getting all files from '{filepath}'")

file_path_list = [filepath + x for x in os.listdir(filepath) if x.endswith(".csv")]
print(f"{len(file_path_list)} files detected")

Getting all files from '/home/workspace/event_data/'
30 files detected


## 3. Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow([
        'artist', 'first_name', 'gender', 'item_in_session', 'last_name', 'length',
        'level', 'location', 'session_id', 'song', 'user_id'
    ])
    for row in tqdm(full_data_rows_list, desc="rows", ncols=100):
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

rows: 100%|█████████████████████████████████████████████████| 8056/8056 [00:00<00:00, 214391.03it/s]


<div class="alert alert-warning" role="alert">
    <b>WARNING:</b> It is a bad idea to join all data and then insert it to Cassandra. It would be better to insert in the first reading if it was a real case. Only doing it for as an example
</div>

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Cassandra Implementation

## 1. Setting up Cassandra
### 1.1. Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

### 1.2. Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS dend_p2 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

### 1.3. Set Keyspace

In [7]:
session.set_keyspace('dend_p2')

## 2. Create queries to ask the following three questions of the data

### 2.1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

```sql
SELECT artist, song, length FROM music_by_session_and_item WHERE session_id = 338 AND item_in_session = 4
```

### 2.2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```sql
SELECT artist, song, first_name, last_name FROM music_by_session WHERE user_id = 10 AND session_id = 182
```

### 2.3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```sql
SELECT first_name, last_name FROM music_by_song WHERE song = 'All Hands Against His Own'
```

## 3. Define and create the tables

In [8]:
tables = {
    "music_by_session_and_item": {
        "cols": "session_id INT, item_in_session INT, artist TEXT, song TEXT, length FLOAT",
        "pkey": "session_id, item_in_session",
    },
    "music_by_session": {
        "cols": "user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, first_name TEXT, last_name TEXT",
        "pkey": "user_id, session_id, item_in_session",
    },
    "music_by_song": {
        "cols": "song TEXT, user_id INT, first_name TEXT, last_name TEXT",
        "pkey": "song, user_id",
    },
}

In [9]:
query = "CREATE TABLE IF NOT EXISTS {} ({}, PRIMARY KEY ({}))"

for table, data in tables.items():
    session.execute(query.format(table, data["cols"], data["pkey"]))

In [10]:
all_cols = [
    "artist",
    "first_name",
    "gender",
    "item_in_session",
    "last_name",
    "length",
    "level",
    "location",
    "session_id",
    "song",
    "user_id"
]
cols_dict = {x: i for i, x in enumerate(all_cols)}

## 4. Insert data

<div class="alert alert-info" role="alert">
    <b>NOTE:</b> The are <b>only</b> less than 10.000 rows so I will use Pandas. This way I do not need to cast numbers prior to insertion
</div>

In [11]:
df = pd.read_csv("event_datafile_new.csv")

query_raw = "INSERT INTO {name} ({columns}) VALUES ({data})"

for name, data in tables.items():
    
    # A list of the columns present in the current table
    cols = [x for x in cols_dict if x in data["cols"]]

    # Create the actual query using the table dict and the query_raw
    query = query_raw.format(
        name=name,
        columns=", ".join(cols),
        data=", ".join(["%s"]*len(cols))
    )
    
    for row in tqdm(df.itertuples(), desc=f"{name:25}", total=df.shape[0], ncols=100):
        # Create a list with all data to insert
        values = [row[cols_dict[x] + 1] for x in cols]
        session.execute(query, values)

music_by_song            : 100%|███████████████████████████████| 6820/6820 [00:07<00:00, 884.38it/s]


## 5. Query data

<div class="alert alert-info" role="alert">
    <b>NOTE:</b> Since the data is so small I will check each query against the pandas dataframe
</div>

To display the results I think is better to create a `pandas` dataframe so that they are rendered as `html` tables.

### 5.1. Query 1

In [12]:
query = "SELECT artist, song, length FROM music_by_session_and_item WHERE session_id = 338 AND item_in_session = 4"
pd.DataFrame([x for x in session.execute(query)])

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Check results

In [13]:
df[(df["session_id"] == 338) & (df["item_in_session"] == 4)]

,artist,first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id
5810,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### 5.2. Query 2

In [14]:
query = "SELECT artist, song, first_name, last_name FROM music_by_session WHERE user_id = 10 AND session_id = 182"
pd.DataFrame([x for x in session.execute(query)])

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Check results

In [15]:
df[(df["user_id"] == 10) & (df["session_id"] == 182)]

,artist,first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id
6805,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
6806,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
6807,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
6808,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


### 5.3. Query 3

In [16]:
query = "SELECT first_name, last_name FROM music_by_song WHERE song = 'All Hands Against His Own'"
pd.DataFrame([x for x in session.execute(query)])

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


#### Check results

In [17]:
df[df["song"] == 'All Hands Against His Own']

,artist,first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id
1313,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
1839,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5282,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


## 6. Drop the tables before closing out the sessions

In [18]:
for table in tables:
    session.execute(f"DROP TABLE IF EXISTS {table}")

## 7. Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()